#1. Cost에 log를 적용하는 이유는 뭔지

##1.1 R

##1.2 python

##2. Dataset 이슈들

- components.csv에 4개 elements가 존재하는 이슈 Row 있음 : 해결됨
- comp_other.csv에 문제있다는데 확인되지 않음
- comp_threaded.csv내 normal_size_1에 See drawing 있음 : 확인필요

#3. Keras : deep learning

http://keras.io/#keras-theano-based-deep-learning-library

#4. RMSLE 기반의 Grid Search CV 최적화

[Kaggle : RMSLE score of 1.68537: How to interpret ??](https://www.kaggle.com/forums/f/15/kaggle-forum/t/9933/rmsle-score-of-1-68537-how-to-interpret)

[stackoverflow : RMSLE 최적화](http://stackoverflow.com/questions/27753775/gbrt-hyperparameter-tuning-using-gridsearchcv)

##Question
I'm running GridSearchCV to find the best parameters for GradientBoostingRegressor.
The tutorial given was to use MSE for scoring.

gs_cv = GridSearchCV(est, param_grid, scoring='mean_squared_error', n_jobs=4).fit(X_train, y_train)
Is it possible to use other own defined scoring such as Root Mean Squared Logarithmic Error (RMSLE) to get the best hyperparameters?

def rmsle(predicted, actual, size):
    return np.sqrt(np.nansum(np.square(np.log(predicted + 1) - np.log(actual + 1)))/float(size))


##Answer
You need to make a custom scorer. In your case it would look like this:

<pre>
    from sklearn.metrics import make_scorer

    scorer = make_scorer(rmsle, greater_is_better=False, size=10)
    grid = GridSearchCV(est, param_grid, scoring=scorer)
    
</pre>

shareedit
answered Jan 3 at 14:47

elyase
12.2k11539
  	 	
Awesome, thanks! –  ananuc Jan 5 at 10:37

#5. tube processing 참고

<pre>
import numpy as np
import pandas as pd


if __name__ == '__main__':
    tube = pd.read_csv('../input/tube.csv',
                       index_col=0,
                       true_values=['Y'],
                       false_values=['N'])

    materials = pd.read_csv('../input/bill_of_materials.csv', index_col=0)
    specs = pd.read_csv('../input/specs.csv', index_col=0)

    for idx in tube.index:
        if tube.ix[idx, 'material_id'] is not np.nan:
            tube.ix[idx, tube.ix[idx, 'material_id']] = 1
        for i in range(1, 11):
            if i < 9 and materials.ix[idx, 'component_id_%d' % i] is not np.nan:
                tube.ix[idx, materials.ix[idx, 'component_id_%d' % i]] = \
                    materials.ix[idx, 'quantity_%d' % i]
            if specs.ix[idx, 'spec%d' % i] is not np.nan:
                tube.ix[idx, specs.ix[idx, 'spec%d' % i]] = 1
    tube.drop('material_id', inplace=True, axis=1)
    tube.fillna(0, inplace=True)
    </pre>

#6. 0.2748 w/ RF

<pre>
options(scipen = 10)

###
### Build train and test db
###

### Load train and test
test = read.csv("../input/test_set.csv")
train = read.csv("../input/train_set.csv")

train$id = -(1:nrow(train))
test$cost = 0

train = rbind(train, test)

### Merge datasets if only 1 variable in common
continueLoop = TRUE
while(continueLoop){
  continueLoop = FALSE
  for(f in dir("../input/")){
    d = read.csv(paste0("../input/", f))
    commonVariables = intersect(names(train), names(d))
    if(length(commonVariables) == 1){
      train = merge(train, d, by = commonVariables, all.x = TRUE)
      continueLoop = TRUE
      print(dim(train))
    }
  }
}

### Clean NA values
for(i in 1:ncol(train)){
  if(is.numeric(train[,i])){
    train[is.na(train[,i]),i] = -1
  }else{
    train[,i] = as.character(train[,i])
    train[is.na(train[,i]),i] = "NAvalue"
    train[,i] = as.factor(train[,i])
  }
}

### Clean variables with too many categories
for(i in 1:ncol(train)){
  if(!is.numeric(train[,i])){
    freq = data.frame(table(train[,i]))
    freq = freq[order(freq$Freq, decreasing = TRUE),]
    train[,i] = as.character(match(train[,i], freq$Var1[1:30]))
    train[is.na(train[,i]),i] = "rareValue"
    train[,i] = as.factor(train[,i])
  }
}

test = train[which(train$id > 0),]
train = train[which(train$id < 0),]

###
### Evaluate RF predictions by splitting the train db in 80%/20%
###

### Randomforest
library(randomForest)

# dtrain_cv = train[which(train$id %% 5 > 0),]
# dtest_cv = train[which(train$id %% 5 == 0),]
# 
# ### Train randomForest on dtrain_cv and evaluate predictions on dtest_cv
# set.seed(123)
# rf1 = randomForest(dtrain_cv$cost~., dtrain_cv[,-match(c("id", "cost"), names(dtrain_cv))], ntree = 10, do.trace = 2)
# 
# pred = predict(rf1, dtest_cv)
# sqrt(mean((log(dtest_cv$cost + 1) - log(pred + 1))^2)) # 0.2589951
# 
# ### With log transformation trick
# set.seed(123)
# rf2 = randomForest(log(dtrain_cv$cost + 1)~., dtrain_cv[,-match(c("id", "cost"), names(dtrain_cv))], ntree = 10, do.trace = 2)
# pred = exp(predict(rf2, dtest_cv)) - 1
# 
# sqrt(mean((log(dtest_cv$cost + 1) - log(pred + 1))^2)) # 0.2410004

### Train randomForest on the whole training set
rf = randomForest(log(train$cost + 1)~., train[,-match(c("id", "cost"), names(train))], ntree = 20, do.trace = 2)

pred = exp(predict(rf, test)) - 1

submitDb = data.frame(id = test$id, cost = pred)
submitDb = aggregate(data.frame(cost = submitDb$cost), by = list(id = submitDb$id), mean)

write.csv(submitDb, "submit.csv", row.names = FALSE, quote = FALSE)
</pre>